In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [7]:
# first import names.csv file and turn into a list of names
import csv

with open('names.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the first row
    with open('names.txt', 'w') as txtfile:
        for i, row in enumerate(csvreader):
            if i < 1000:  # for the first 1000 rows
                name1 = row[1]
                name2 = row[2]
                txtfile.write(name1 + '\n')
                txtfile.write(name2 + '\n') 

In [8]:
# turn all letters to lowercase. this is important because 'A' and 'a' are different characters for the model
with open('names.txt', 'r') as f:
    names = f.read()

with open('names.txt', 'w') as f:
    f.write(names.lower())

In [9]:
# we can now read the names.txt file
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(words[:10])

2000
['olivia', 'liam', 'emma', 'noah', 'charlotte', 'oliver', 'amelia', 'elijah', 'ava', 'james']


In [10]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [12]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([13643, 3]), torch.int64, torch.Size([13643]), torch.int64)

In [13]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([10920, 3]) torch.Size([10920])
torch.Size([1354, 3]) torch.Size([1354])
torch.Size([1369, 3]) torch.Size([1369])


In [14]:
C = torch.randn((27, 2))
emb = C[X]
emb.shape
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.0909, -0.9249,  0.2093,  ...,  0.9847,  0.9497, -0.4557],
        [ 0.8935, -0.9865,  0.2648,  ...,  0.9983,  0.9148,  0.2228],
        [-0.4260, -0.6965, -0.5236,  ...,  0.9980,  0.9967,  0.3175],
        ...,
        [-1.0000,  1.0000, -0.9958,  ...,  0.8040,  1.0000, -0.0502],
        [ 0.7736, -1.0000,  0.9990,  ...,  0.9781, -0.9899, -0.9407],
        [ 0.8168, -0.3131, -0.5392,  ...,  0.5724,  0.3292, -0.7148]])